In [1]:
import pandas as pd
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw
from segmentation import segment

df = pd.read_csv('E:/LUNA/annotation/annotations_with_malignancy.csv')
df.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm,mal_bool,mal_details,bboxLowX,bboxLowY,bboxLowZ,bboxHighX,bboxHighY,bboxHighZ,len_mal_details
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471,True,"[4, 2, 4, 2]",-131.896480,-178.259761,-299.800004,-125.451167,-172.458980,-296.200004,4
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735...,-69.175741,218.052625,-668.501273,4.441200,False,"[3, 2, 3]",-71.815939,215.709820,-670.900024,-66.722189,220.803570,-666.900024,3
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-67.459503,-247.744211,-211.095340,7.744222,True,"[5, 3, 4]",-73.027344,-253.847656,-213.100004,-61.074219,-242.558594,-209.500004,3
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-97.690876,-134.068054,-187.094883,6.690036,False,"[3, 3, 3]",-102.910156,-137.636719,-189.700005,-92.949219,-130.332031,-186.100005,3
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414...,69.529141,87.036153,-149.190113,10.824515,False,"[3, 3, 5, 2]",63.731247,80.468750,-154.000000,76.231247,93.750000,-144.000000,4


In [31]:
candidate = pd.read_csv('E:/LUNA/annotation/candidates_V2.csv')
candidate.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


In [32]:
unique_count = candidate['seriesuid'].nunique()
unique_count

888

In [44]:
import os
import csv
import collections
from collections import namedtuple  
mhd_list = glob.glob('E:/LUNA/Luna_Data/subset*/*.mhd')
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm, series_uid, center_xyz')
requireOnDisk_bool=True
presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

candidateInfo_list = []
with open('E:/LUNA/annotation/annotations_with_malignancy.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
        annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
        annotationDiameter_mm = float(row[4])
        isMal_bool = {'False': False, 'True': True}[row[5]] #it record the malignancy or not

        if series_uid not in presentOnDisk_set and requireOnDisk_bool:
            continue

        candidateInfo_list.append(
            CandidateInfoTuple(
                True,
                True,
                isMal_bool,
                annotationDiameter_mm,
                series_uid,
                annotationCenter_xyz,
            )
        )

with open('E:/LUNA/annotation/candidates_V2.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]

        if series_uid not in presentOnDisk_set and requireOnDisk_bool:
            continue

        isNodule_bool = bool(int(row[4]))
        candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

        if not isNodule_bool:
            candidateInfo_list.append(
                CandidateInfoTuple(
                    False,
                    False,
                    False,
                    0.0,
                    series_uid,
                    candidateCenter_xyz,
                )
            )

candidateInfo_list.sort(reverse=True)

In [45]:
candidateInfo_dict = {}

for candidateInfo_tup in candidateInfo_list:
    candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                  []).append(candidateInfo_tup)

In [46]:
len(candidateInfo_dict)

888

In [47]:
candidateInfo_list.sort(key=lambda x: x.series_uid)

In [48]:
len(candidateInfo_dict)

888

In [3]:
tmp = candidate['class']
tmp.value_counts()

class
0    753418
1      1557
Name: count, dtype: int64

In [4]:
tmp_data = []
for index, row in df.iterrows():
    seriesuid = row['seriesuid']
    coordX = row['coordX']
    coordY = row['coordY']
    coordZ = row['coordZ']
    mal_bool = row['mal_bool']
    
    
    # Append a new row to the new_data list
    tmp_data.append([seriesuid, coordX, coordY, coordZ, mal_bool])

# Create a new DataFrame with the specified columns
new_df = pd.DataFrame(tmp_data, columns=['seriesuid', 'coordX', 'coordY', 'coordZ', 'mal_bool'])

# Print the new DataFrame
new_df

,seriesuid,coordX,coordY,coordZ,mal_bool
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,True
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735...,-69.175741,218.052625,-668.501273,False
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-67.459503,-247.744211,-211.095340,True
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-97.690876,-134.068054,-187.094883,False
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414...,69.529141,87.036153,-149.190113,False
...,...,...,...,...,...
1177,1.3.6.1.4.1.14519.5.2.1.6279.6001.299767339686...,-57.725518,73.779494,-147.415580,False
1178,1.3.6.1.4.1.14519.5.2.1.6279.6001.299767339686...,-72.287990,-77.481968,-160.486254,False
1179,1.3.6.1.4.1.14519.5.2.1.6279.6001.299767339686...,96.625954,-46.154823,-88.229432,False
1180,1.3.6.1.4.1.14519.5.2.1.6279.6001.297433269262...,33.104795,-63.116434,-108.725571,False


In [5]:
unique_value_count = new_df['coordZ'].nunique()
print(unique_value_count)

1182


In [6]:
# Create an empty dictionary to store the result
result_dict = {}

# Iterate through the DataFrame and populate the dictionary
for index, row in df.iterrows():
    seriesuid = row['seriesuid']
    coordinates = [row['coordX'], row['coordY'], row['coordZ']]
    # Check if the seriesuid is already in the dictionary, if not, add a new entry
    if seriesuid in result_dict:
        result_dict[seriesuid].append(coordinates)
    else:
        result_dict[seriesuid] = [coordinates]

# Print the resulting dictionary
# print(result_dict)

In [7]:
# # Iterate through the dictionary and print key along with the count of coordinate sets
# for key, value in result_dict.items():
#     print(f"{key}: {len(value)}")

In [8]:
len(result_dict)

600

In [9]:
# for key in result_dict.keys():
#     print(key, result_dict[key])

In [10]:
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


In [11]:
from tqdm import tqdm

In [12]:
list(result_dict.keys())[0:10]

['1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735520890308424143898',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002726404418399209377',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414087561012670296643',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.277452631455527999380186898011',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.277662902666135640561346462196',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.278010349511857248000260557753',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.279300249795483097365868125932',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478',
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.280125803152924778388346920341']

In [13]:
def np2Png(np_arr, target_name):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slice_ori = Image.fromarray(scaled_np_arr, mode='L')
    slice_ori.save(target_name)

In [14]:
final_list = []
uid_list = []
nob_list = []
boolean_list = []
counter = 0
for key in tqdm(list(result_dict.keys()), desc="Searching"):
    # print(key, result_dict[key])
    # Glob finds all the pathnames matching a specified pattern.
    # In this case, it finds a file with the given format and assigns the path to `mhd_path`.
    mhd_path = glob.glob('E:/LUNA/Luna_Data/subset*/{}.mhd'.format(key))
    # print(mhd_path)
    
    # Read the medical image from the specified path using SimpleITK.
    ct_mhd = sitk.ReadImage(mhd_path)
    # print(ct_mhd)
    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]
    # Convert the SimpleITK image to a NumPy array.
    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    # print('hu_a',hu_a)
    
    # # Get the origin (starting point) of the image in XYZ coordinates.
    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

    # # # Get the voxel size in each dimension (X, Y, Z) from the image spacing.
    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

    # # # Get the direction matrix that defines the image orientation.
    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    # add for loop right here
    # converts XYZ coordinates to IRC (Index, Row, Column) coordinates.
    for i in range(len(result_dict[key])):
        center_xyz = result_dict[key][i]
        # print('i =', i)
        # print(center_xyz)
        
        # Find the true or false of the patient's nodule
        boolean = new_df.loc[new_df['coordX'] == center_xyz[0], 'mal_bool'].values[0]
        boolean_list.append(boolean)
        uid_list.append(key)
        nob_list.append(i)
        # print('boolean =', boolean)
        # print('===========')
        center_irc = xyz2irc(center_xyz, origin_xyz, vxSize_xyz, direction_a)
        
        # Print the calculated `center_irc`.
        # print(center_irc)
        # index, row, column
        index = center_irc[0]
        row = center_irc[1]
        col = center_irc[2]
        # print('original index =', index)
        
        # print(hu_a[index].shape)
        # print('index=', hu_a[index])
        final_list.append(hu_a[index])
        
    if ((len(final_list)) > 100):
        print(len(final_list))
        stack_list = np.stack(final_list)
        result = segment(stack_list)
        print(stack_list.shape)
        for j in range(0, len(final_list)):
            np2Png(result[j].astype(float), "./yolo/segmented/{}_{}_{}_{}.png".format(j + counter, boolean_list[j], nob_list[j], uid_list[j]))
        counter += len(final_list)
        final_list = []
        boolean_list = []
        nob_list = []
        uid_list = []
        
        # Calculate the minimum and maximum values in the 78th slice of the CT image.
        # min_value = np.min(hu_a[index])
        # max_value = np.max(hu_a[index])
        # print('min =', min_value)
        # print('max =', max_value)

        # # Scale the pixel values of the 78th slice to the range [0, 255].
        # scaled_hu_a = (hu_a[index] - min_value) / (max_value - min_value) * 255

        # # Convert the scaled array to an integer data type (uint8).
        # scaled_hu_a = scaled_hu_a.astype(np.uint8)
        # print('scaled_hu_a =', scaled_hu_a)
        
        # # # 組合文件名，使用format方法將key和i插入文件名中
        # # filename = "./origin_data/marked_{}_{}_{}.png".format(boolean, key, i)
        # filename = "origin_{}_{}_{}.png".format(key, i, boolean)
        # slice_image = Image.fromarray(scaled_hu_a, mode='L')
        # # # 保存圖片，文件名使用組合的filename
        # slice_image.save(filename)

        # # =================================================
        
        # # add mark
        # # Create a drawing context on the image
        # draw = ImageDraw.Draw(slice_image)

        # # Define the coordinates to mark (row 212, column 45) as a red rectangle
        # x1, y1, x2, y2 = col-10, row-10, col+10, row+10  # Adjust these coordinates as needed
        # # print(x1, x2, y1, y2)
        # # Define the outline color
        # outline_color = (0, 0, 255)  

        # # Draw a white rectangle on the image to mark the specific row and column
        # draw.rectangle([x1, y1, x2, y2], outline="white", width=1)  # Increase width for better visibility

        # # Save the marked slice as a PNG
        # mark_filename = "marked_{}_{}_{}.png".format(key, i, boolean)
        # # 保存圖片，文件名使用組合的filename
        # slice_image.save(mark_filename)
        # # print('=============')

Searching:   8%|█████▎                                                                | 45/600 [00:13<02:13,  4.15it/s]

102


C:\Users\oplab\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\oplab\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(102, 512, 512)


Searching:  15%|██████████▌                                                           | 90/600 [03:44<02:32,  3.34it/s]

...

Searching:  84%|████████████████████████████████████████████████████████           | 502/600 [1:14:26<09:27,  5.79s/it]

101
(101, 512, 512)


Searching:  93%|██████████████████████████████████████████████████████████████     | 556/600 [1:29:19<26:45, 36.50s/it]

109
(109, 512, 512)


Searching: 100%|███████████████████████████████████████████████████████████████████| 600/600 [1:38:02<00:00,  9.80s/it]


In [15]:
# uid_list

In [16]:
# nob_list

In [17]:
# boolean_list

In [25]:
print(len(final_list))

53


In [18]:
# print(stack_list)

In [19]:
from segmentation import segment

In [26]:
stack_list = np.stack(final_list)
result = segment(stack_list)
print(result.shape)

(53, 512, 512)


In [27]:
print(counter)

1129


In [28]:
for i in range(0, 53):
    np2Png(result[i].astype(float), "./yolo/segmented/{}_{}_{}_{}.png".format(i + counter, boolean_list[i], nob_list[i], uid_list[i]))

In [30]:
final_list = []
counter = 0
for key in tqdm(list(result_dict.keys()), desc="Searching"):
    # print(key, result_dict[key])
    # Glob finds all the pathnames matching a specified pattern.
    # In this case, it finds a file with the given format and assigns the path to `mhd_path`.
    mhd_path = glob.glob('E:/LUNA/Luna_Data/subset*/{}.mhd'.format(key))
    # print(mhd_path)
    
    # Read the medical image from the specified path using SimpleITK.
    ct_mhd = sitk.ReadImage(mhd_path)
    # print(ct_mhd)
    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]
    # Convert the SimpleITK image to a NumPy array.
    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    # print('hu_a',hu_a)
    
    # # Get the origin (starting point) of the image in XYZ coordinates.
    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

    # # # Get the voxel size in each dimension (X, Y, Z) from the image spacing.
    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

    # # # Get the direction matrix that defines the image orientation.
    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    # add for loop right here
    # converts XYZ coordinates to IRC (Index, Row, Column) coordinates.
    for i in range(len(result_dict[key])):
        center_xyz = result_dict[key][i]
        # print('i =', i)
        # print(center_xyz)
        
        # Find the true or false of the patient's nodule
        boolean = new_df.loc[new_df['coordX'] == center_xyz[0], 'mal_bool'].values[0]
        # print('boolean =', boolean)
        # print('===========')
        center_irc = xyz2irc(center_xyz, origin_xyz, vxSize_xyz, direction_a)
        
        # Print the calculated `center_irc`.
        # print(center_irc)
        # index, row, column
        index = center_irc[0]
        row = center_irc[1]
        col = center_irc[2]
        # print('original index =', index)
        
        # print(hu_a[index].shape)
        # print('index=', hu_a[index])
        
        # Calculate the minimum and maximum values in the 78th slice of the CT image.
        min_value = np.min(hu_a[index])
        max_value = np.max(hu_a[index])
        # print('min =', min_value)
        # print('max =', max_value)

        # # Scale the pixel values of the 78th slice to the range [0, 255].
        scaled_hu_a = (hu_a[index] - min_value) / (max_value - min_value) * 255

        # # Convert the scaled array to an integer data type (uint8).
        scaled_hu_a = scaled_hu_a.astype(np.uint8)
        # print('scaled_hu_a =', scaled_hu_a)
        
        # # # 組合文件名，使用format方法將key和i插入文件名中
        # filename = "./origin_data/marked_{}_{}_{}.png".format(boolean, key, i)
        path = "./yolo/origin/{}_{}_{}_{}.png".format(counter, boolean, i, key)
        slice_image = Image.fromarray(scaled_hu_a, mode='L')
        # # 保存圖片，文件名使用組合的filename
        slice_image.save(path)

        # =================================================
        
        # add mark
        # Create a drawing context on the image
        draw = ImageDraw.Draw(slice_image)

        # Define the coordinates to mark (row 212, column 45) as a red rectangle
        x1, y1, x2, y2 = col-10, row-10, col+10, row+10  # Adjust these coordinates as needed
        # print(x1, x2, y1, y2)
        # Define the outline color
        outline_color = (0, 0, 255)  

        # Draw a white rectangle on the image to mark the specific row and column
        draw.rectangle([x1, y1, x2, y2], outline="white", width=1)  # Increase width for better visibility

        # Save the marked slice as a PNG
        marked_path = "./yolo/marked/{}_{}_{}_{}.png".format(counter, boolean, i, key)
        # 保存圖片，文件名使用組合的filename
        slice_image.save(marked_path)
        counter += 1
        # print('=============')

Searching: 100%|███████████████████████████████████████████████████████████████████| 600/600 [2:02:12<00:00, 12.22s/it]


In [22]:
# import os

# origindir = "E:\\LUNA\\yolo\\origin_data"
# targetdirs = ["E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\train\\images", "E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\test\\images", "E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\valid\\images"]

# ori_files = os.listdir(origindir)

# target_files = []
# for targetdir in targetdirs:
#     target_files.append(os.listdir(targetdir))

# for file in ori_files:
#     last_dot_index = file.rfind(".")
#     prefix = ""
#     if last_dot_index != -1:
#         # Split the string into two parts: everything before the last dot and the file extension
#         before_last_dot = file[:last_dot_index]
#         file_extension = file[last_dot_index + 1:]

#         # Replace the last dot with an underscore
#         prefix = before_last_dot + "_" + file_extension
#         prefix = prefix.replace(".", "-")
#         print(prefix)
#     for file in target_files[1]:
#         if file.startswith(prefix):
#             # You've found a file with the desired prefix
#             print(f"Found file with prefix '{prefix}': {file}")

In [23]:
# # Find the last dot (.) in the string
# last_dot_index = input_string.rfind(".")

# if last_dot_index != -1:
#     # Split the string into two parts: everything before the last dot and the file extension
#     before_last_dot = input_string[:last_dot_index]
#     file_extension = input_string[last_dot_index + 1:]

#     # Replace the last dot with an underscore
#     modified_string = before_last_dot + "_" + file_extension

#     print(modified_string)
# else:
#     # If there is no dot in the string, do something else or handle the case as needed
#     print("No dot found in the input string")